In [53]:
import oracledb, os, time, pandas as pd

In [12]:
iot_telemetry = pd.read_csv("data/iot_telemetry_data.csv")
iot_telemetry = iot_telemetry.dropna()
iot_telemetry.head()

,ts,device,co,humidity,light,lpg,motion,smoke,temp
0,1.594512e+09,b8:27:eb:bf:9d:51,0.004956,51.000000,False,0.007651,False,0.020411,22.700000
1,1.594512e+09,00:0f:00:70:91:0a,0.002840,76.000000,False,0.005114,False,0.013275,19.700001
2,1.594512e+09,b8:27:eb:bf:9d:51,0.004976,50.900000,False,0.007673,False,0.020475,22.600000
3,1.594512e+09,1c:bf:ce:15:ec:4d,0.004403,76.800003,True,0.007023,False,0.018628,27.000000
4,1.594512e+09,b8:27:eb:bf:9d:51,0.004967,50.900000,False,0.007664,False,0.020448,22.600000


In [21]:
print(len("1c:bf:ce:15:ec:4d"))

17


# Initializing OracleDB

In [16]:
try:
    connection = oracledb.connect(
            user="SYSTEM",
            password=os.getenv("ORACLE_PASSWORD"), 
            dsn="localhost/FREEPDB1"
        )
    print("Verbindung erfolgreich!")
except oracledb.DatabaseError as e:
    print(f"Verbindung fehlgeschlagen! {e}")

Verbindung erfolgreich!


In [48]:
# Create table query
create_table_query = """
CREATE TABLE IOT_TELEMETRY_DATA (
    IOT_TELEMETRY_ID  NUMBER GENERATED BY DEFAULT ON NULL AS IDENTITY NOT NULL,
    TS                TIMESTAMP        NOT NULL,
    DEVICE            VARCHAR2(17)  NOT NULL,
    CO                NUMBER,
    HUMIDITY          NUMBER,
    LIGHT             VARCHAR(5),
    LPG               NUMBER,
    MOTION            VARCHAR(5),
    SMOKE             NUMBER,
    TEMP              NUMBER
) 
"""

In [49]:
try:
    cursor = connection.cursor()
    cursor.execute(create_table_query)
    print("Tabelle erfolgreich erstellt!")
except oracledb.DatabaseError as e:
    print(f"Tabelle konnte nicht erstellt werden! {e}")

Tabelle erfolgreich erstellt!


# Insert  to Database

In [50]:
# Insert data into the table from df iot_telemetry
# Convert the 'TS' column to a timestamp format
iot_telemetry['ts'] = pd.to_datetime(iot_telemetry['ts'], unit='s')
iot_telemetry['light'] = iot_telemetry['light'].astype(str)
iot_telemetry['motion'] = iot_telemetry['motion'].astype(str)

# Convert the DataFrame to a list of tuples
data_tuples = [tuple(x) for x in iot_telemetry.to_numpy()]
# Insert the data into the table
insert_query = """
INSERT INTO IOT_TELEMETRY_DATA (TS, DEVICE, CO, HUMIDITY, LIGHT, LPG, MOTION, SMOKE, TEMP)
VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9)
"""

In [51]:
# Print the first 5 rows of the data tuple
for i in range(5):
    print(data_tuples[i])

(Timestamp('2020-07-12 00:01:34.385974646'), 'b8:27:eb:bf:9d:51', 0.0049559386483912, 51.0, 'False', 0.0076508222705571, 'False', 0.0204112701224129, 22.7)
(Timestamp('2020-07-12 00:01:34.735567570'), '00:0f:00:70:91:0a', 0.0028400886071015, 76.0, 'False', 0.005114383400977, 'False', 0.0132748367048515, 19.700000762939453)
(Timestamp('2020-07-12 00:01:38.073572874'), 'b8:27:eb:bf:9d:51', 0.0049760123404216, 50.9, 'False', 0.007673227406398, 'False', 0.0204751255761782, 22.6)
(Timestamp('2020-07-12 00:01:39.589145899'), '1c:bf:ce:15:ec:4d', 0.0044030268296996, 76.80000305175781, 'True', 0.0070233371458773, 'False', 0.0186282253770188, 27.0)
(Timestamp('2020-07-12 00:01:41.761234999'), 'b8:27:eb:bf:9d:51', 0.0049673636419089, 50.9, 'False', 0.0076635772823724, 'False', 0.0204476208102336, 22.6)


In [52]:
try:
    cursor.executemany(insert_query, data_tuples)
    connection.commit()
    print("Data inserted successfully!")
except oracledb.DatabaseError as e:
    print("Error inserting data:", e)

Data inserted successfully!


In [57]:
# Reading all data from the table
read_query = "SELECT * FROM IOT_TELEMETRY_DATA"
try:
    start = time.time()
    cursor.execute(read_query)
    rows = cursor.fetchall()
    end = time.time()
    for i in range(1, 5):
        print(rows[i])
    print(f"Data read successfully in {end - start:.2f} seconds!")
    print(len(rows))
except oracledb.DatabaseError as e:
    print("Error reading data:", e)

(2, datetime.datetime(2020, 7, 12, 0, 1, 34), '00:0f:00:70:91:0a', 0.0028400886071015, 76, 'False', 0.005114383400977, 'False', 0.0132748367048515, 19.700000762939453)
(3, datetime.datetime(2020, 7, 12, 0, 1, 38), 'b8:27:eb:bf:9d:51', 0.0049760123404216, 50.9, 'False', 0.007673227406398, 'False', 0.0204751255761782, 22.6)
(4, datetime.datetime(2020, 7, 12, 0, 1, 39), '1c:bf:ce:15:ec:4d', 0.0044030268296996, 76.80000305175781, 'True', 0.0070233371458773, 'False', 0.0186282253770188, 27)
(5, datetime.datetime(2020, 7, 12, 0, 1, 41), 'b8:27:eb:bf:9d:51', 0.0049673636419089, 50.9, 'False', 0.0076635772823724, 'False', 0.0204476208102336, 22.6)
Data read successfully in 4.17 seconds!
405184
